**Main Game**

In [16]:
import pygame
import random
import sys

# Initialize Pygame
pygame.init()

# Set up display
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Geometry Slide")

# Colors
white = (255, 255, 255)
black = (0, 0, 0)

# Player settings
player_size = 50
player_x = width // 2
player_y = 400
player_speed = 5
player = pygame.Rect(player_x, player_y, player_size, player_size)
jump_strength = 25
can_jump = False
coyote_time = 0

# Platform settings
platform_start_y = 200
platform_width = 90
platform_height = 40

platform_1 = pygame.Rect((width+platform_width)/3, platform_start_y, platform_width, platform_height)
platform_2 = pygame.Rect(2*(width+platform_width)/3, platform_start_y, platform_width, platform_height)
platform_3 = pygame.Rect((width+platform_width), platform_start_y, platform_width, platform_height)
platforms = [platform_1, platform_2, platform_3]

start_platform_left = False
start_platform = pygame.Rect(300, 500, 200, 20)

platform_velocity = 2

last_y = platform_start_y

# Gravity settings
gravity = 0.9816
player_velocity_y = 0
max_fall_speed = 100

#Create score
score = 0
font = pygame.font.Font(None, 36)
last_platform = None

# Create a clock object to control the frame rate
clock = pygame.time.Clock()
fps = 60  # Set the desired frames per second

player_image = pygame.image.load('Player_Yoshi.png')  # Load the player image
player_image = pygame.transform.scale(player_image, (player_size, player_size))

background_image = pygame.image.load('Background_Sky.jpg')  # Load the background image
background_image = pygame.transform.scale(background_image, (width, height))  # Scale it to fit the window

platform_image = pygame.image.load('Platform_Cloud.png')  # Load the platform image
platform_image = pygame.transform.scale(platform_image, (platform_width, platform_height))  # Scale it to fit the platform

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Fill the screen with white
    screen.blit(background_image, (0, 0))

    # Player movement
    keys = pygame.key.get_pressed()
    if keys[pygame.K_a]:
        player.x -= player_speed
    if keys[pygame.K_d]:
        player.x += player_speed


    # Check for jump
    coyote_time -= fps/20
    if keys[pygame.K_SPACE] and can_jump and coyote_time >= 0:
        player_velocity_y = -jump_strength
        can_jump = False
        if start_platform_left == False:
            start_platform_left = True

    # Apply gravity
    player_velocity_y += gravity
    if player_velocity_y > max_fall_speed:
        player_velocity_y = max_fall_speed

    player.y += player_velocity_y
    if player.y >= height + player_size + 10:
        running = False

    # Collision detection with platforms

    if player.colliderect(start_platform) and player_velocity_y > 0 and not start_platform_left:
            player.y = start_platform.y - player.height
            player_velocity_y = 0
            can_jump = True
            coyote_time = 0.5*fps

    for platform in platforms:
        if player.colliderect(platform) and player_velocity_y > 0:
            #Player collision mechanics
            player.y = platform.y - player.height
            player_velocity_y = 0
            player.x -= platform_velocity

            #Jump mechanics
            can_jump = True
            coyote_time = 0.5*fps

            if platform != last_platform:
                score += 1
                last_platform = platform

        # Platform mpvement and spawn
        platform.x -= platform_velocity
        if platform.x < -platform_width:
            platform.x = width
            max_jump_height = jump_strength**2 / (2 * gravity)
            lower_limit = int(max(last_y - max_jump_height + 1, platform_height + player_size))
            platform.y = random.randint(lower_limit, height-platform_height)

            last_y = platform.y
        # Draw platform images
    for platform in platforms:
        screen.blit(platform_image, (platform.x, platform.y))

    if not start_platform_left:
        screen.blit(platform_image, (start_platform.x, start_platform.y))
    else:
        platform_velocity += 0.001

    screen.blit(player_image, (player.x, player.y))

    score_text = font.render(f"Score: {score}", True, black)
    screen.blit(score_text, (10, 10))

    # Update the display
    pygame.display.flip()

    # Cap the frame rate to 60 fps
    clock.tick(fps)

# Quit Pygame
pygame.quit()
sys.exit()


SystemExit: 

**OOC** Object oriented code

In [15]:
import pygame
import random
import sys

class Game:
    def __init__(self):
        pygame.init()
        self.width, self.height = 800, 600
        self.screen = pygame.display.set_mode((self.width, self.height))
        pygame.display.set_caption("Geometry Slide")

        self.white = (255, 255, 255)
        self.black = (0, 0, 0)
        self.blue = (0, 0, 255)

        self.clock = pygame.time.Clock()
        self.fps = 60
        self.running = True

        self.player = Player(self.width // 2, 400, 50, 5, 25)
        self.platforms = [Platform(self.width, 200, 200, 20)]
        self.start_platform = Platform(300, 500, 200, 20)
        self.start_platform_left = False
        self.platform_velocity = 2

        self.intro_text = "Start"

    def run(self):
        while self.running:
            self.handle_events()
            self.update()
            self.draw()

        pygame.quit()
        sys.exit()

    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.running = False

    def update(self):
        keys = pygame.key.get_pressed()
        self.player.move(keys, self.start_platform_left)
        
        if keys[pygame.K_SPACE] and self.player.can_jump and self.player.coyote_time >= 0:
            self.player.jump()
            if not self.start_platform_left:
                self.start_platform_left = True
        
        self.player.apply_gravity()
        self.player.check_fall(self.height)

        self.check_collisions()

        for platform in self.platforms:
            platform.move(self.platform_velocity, self.width)
        
        if self.start_platform_left:
            self.platform_velocity += 0.01

    def check_collisions(self):
        if not self.start_platform_left:
            self.player.check_platform_collision(self.start_platform, True)
        for platform in self.platforms:
            self.player.check_platform_collision(platform, True)

        
    def draw(self):
        self.screen.fill(self.white)

        for platform in self.platforms:
            platform.draw(self.screen, self.black)

        if not self.start_platform_left:
            self.start_platform.draw(self.screen, self.black)

        self.player.draw(self.screen, self.blue)
        
        pygame.display.flip()
        self.clock.tick(self.fps)

class Player:
    def __init__(self, x, y, size, speed, jump_strength):
        self.rect = pygame.Rect(x, y, size, size)
        self.speed = speed
        self.jump_strength = jump_strength
        self.can_jump = False
        self.coyote_time = 0
        self.velocity_y = 0
        self.gravity = 0.9816
        self.max_fall_speed = 100

    def move(self, keys, start_platform_left):
        if keys[pygame.K_a]:
            self.rect.x -= self.speed
        if keys[pygame.K_d]:
            self.rect.x += self.speed
        self.coyote_time -= 60 / 20

    def jump(self):
        self.velocity_y = -self.jump_strength
        self.can_jump = False

    def apply_gravity(self):
        self.velocity_y += self.gravity
        if self.velocity_y > self.max_fall_speed:
            self.velocity_y = self.max_fall_speed
        self.rect.y += self.velocity_y

    def check_fall(self, height):
        if self.rect.y >= height + self.rect.height + 10:
            pygame.quit()
            sys.exit()

    def check_platform_collision(self, platform, active):
        if self.rect.colliderect(platform.rect) and self.velocity_y > 0 and active:
            self.rect.y = platform.rect.y - self.rect.height
            self.velocity_y = 0
            self.can_jump = True
            self.coyote_time = 0.5 * 60


    def draw(self, screen, color):
        pygame.draw.rect(screen, color, self.rect)

class Platform:
    def __init__(self, x, y, width, height):
        self.rect = pygame.Rect(x, y, width, height)

    def move(self, velocity, screen_width):
        self.rect.x -= velocity
        if self.rect.x < -self.rect.width:
            self.rect.x = screen_width
            self.rect.y = random.randint(0, 600)

    def draw(self, screen, color):
        pygame.draw.rect(screen, color, self.rect)

if __name__ == "__main__":
    game = Game()
    game.run()

SystemExit: 

c:\Users\anton\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
